## Gray-scott model

This model can be used as a way to express chemical reaction of two generic substances or diffusion. Usually with chemical reaction:

$$2V+U\rightarrow3V$$

A step further, the model can be expressed as below:

$$\frac{\partial u}{ \partial t} = D_u\Delta u-uv^2+F(1-u)\quad\quad(1)$$

$$\frac{\partial v}{\partial t} = D_v\Delta v + uv^2-(F+k)v\quad\quad(2)$$

For above two equations, I'd like to replace $\nabla^2$ with $Laplace\space operator\space \Delta$ for convenience. $\Delta u$ and $\Delta v$ can be seen as left side of Laplace equations:

$$\Delta u = \frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2}\quad\quad(3)$$

$$\Delta v = \frac{\partial^2 v}{\partial x^2} + \frac{\partial^2 v}{\partial y^2}\quad\quad(4)$$

So this problem is 2D problem. 

In [2]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [6]:
uvinitial = numpy.load('/home/lyapage/Homework_reaction/uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

In [1]:
fig = pyplot.figure(figsize=(8,5))
pyplot.subplot(121)
pyplot.imshow(U, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);
pyplot.subplot(122)
pyplot.imshow(V, cmap = cm.RdBu)
pyplot.xticks([]), pyplot.yticks([]);

NameError: name 'pyplot' is not defined

## Stability

First of all, explicit method will be applied here. Explicit method is not unconditially stable so the first thing I will do here is to check the stability  of this problem:

$$D_u(\frac{\Delta t}{(\Delta x)^2}+\frac{\Delta t}{(\Delta y)^2}) = 2D_u\frac{\Delta t}{\delta^2}  \quad\quad(5)$$

$$D_v(\frac{\Delta t}{(\Delta x)^2}+\frac{\Delta t}{(\Delta y)^2}) = 2D_v\frac{\Delta t}{\delta^2}  \quad\quad(6)$$

In order to make it stable, equation (5) and (6) should be less than $\frac{1}{2}$. 



In [75]:
# Given conditions:
n = 192

Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065

dh = 5/(n-1)

T = 8000.0

dt = .9 * dh**2 / (4*max(Du,Dv))

nt = int(T/dt)

In [76]:
# Now put the given condition into equation (5) and (6)

sigma_u = 2*Du*dt/dh**2    #CFL number for U
sigma_v = 2*Dv*dt/dh**2    #CFL number for V
CFL_stable = 0.5    #obtained from lesson2

if sigma_u < CFL_stable and sigma_v < CFL_stable:
    print('It is stable with given conditions.')
else: 
    print('It is not stable, actually implicit method needed here.')

It is stable with given conditions.


## 2D Finite differences

Equation (1) and (2) can be discretized as numerical form:

$$\frac{U^{n+1}_{i,j}-U_{i,j}^n}{\Delta t} =D_u\Big [ \frac{U^n_{i+1,j}-2U^n_{i,j}+U^n_{i-1,j}}{(\Delta x)^2} + \frac{U^n_{i,j+1}-2U^n_{i,j}+U^n_{i,j-1}}{(\Delta y)^2} \Big ] - U^n_{i,j}(V^n_{i,j})^2 + F(1-U^n_{i,j})\quad\quad(7)$$

$$\frac{V^{n+1}_{i,j}-V_{i,j}^n}{\Delta t} =D_v\Big [ \frac{V^n_{i+1,j}-2V^n_{i,j}+V^n_{i-1,j}}{(\Delta x)^2} + \frac{V^n_{i,j+1}-2V^n_{i,j}+V^n_{i,j-1}}{(\Delta y)^2} \Big ] + U^n_{i,j}(V^n_{i,j})^2 - (F+k)V^n_{i,j}\quad\quad(8)$$

With known condition: $\Delta x=\Delta y =\delta$, transform above two equations into:


$$U^{n+1}_{i,j} = U_{i,j}^n + D_u \frac{\Delta t}{\delta^2}\Big[ U^n_{i+1,j}+U^n_{i-1,j} -4U_{i,j}^n + U_{i,j+1}^n +U_{i,j-1}^n \Big] - \Delta tU^n_{i,j}(V^n_{i,l}) + \Delta t F(1-U_{i,j,}^n)\quad\quad(9)$$

$$V^{n+1}_{i,j} = V_{i,j}^n + D_v \frac{\Delta t}{\delta^2}\Big[ V^n_{i+1,j}+V^n_{i-1,j} -4V_{i,j}^n + V_{i,j+1}^n +V_{i,j-1}^n \Big] + \Delta tU^n_{i,j}(V^n_{i,l}) - \Delta t (F+k)V_{i,j}^n\quad\quad(10)$$



## Boundary condition

All sides satisfy Neumann condition:

$$\frac{\partial U}{\partial x}\Big|_{x=0,5}=0,\frac{\partial U}{\partial y}\Big|_{y=0,5}=0,\frac{\partial V}{\partial x}\Big|_{x=0,5}=0,\frac{\partial V}{\partial y}\Big|_{y=0,5}=0\quad\quad(11)$$

$$\implies Un[0,:] = Un[1,:],Un[-1,:] = Un[-2,:],Un[:,0] = Un[:,1],Un[:,-1] = Un[:,-2]\quad\quad(12)$$

$$\implies Vn[0,:] = Vn[1,:],Vn[-1,:] = Vn[-2,:],Vn[:,0] = Vn[:,1],Vn[:,-1] = Vn[:,-2]\quad\quad(13)$$

In [80]:
def ftcs(U,V,nt,dh,dt,Du,Dv,F,k):
    
    for n in range(nt):
        
        Un = U.copy()
        Vn = V.copy()
        U[1:-1,1:-1] = Un[1:-1,1:-1] + Du*dt/dh**2*(Un[2:,1:-1] + Un[:-2,1:-1] -\
                       4*Un[1:-1,1:-1] + Un[1:-1,2:] + Un[1:-1,:-2]) -\
                       dt*Un[1:-1,1:-1]*Vn[1:-1,1:-1]**2 +\
                       dt*F*(1 - Un[1:-1,1:-1])
        
        V[1:-1,1:-1] = Vn[1:-1,1:-1] + Dv*dt/dh**2*(Vn[2:,1:-1] + Vn[:-2,1:-1] -\
                       4*Vn[1:-1,1:-1] + Vn[1:-1,2:] + Vn[1:-1,:-2]) +\
                       dt*Un[1:-1,1:-1]*Vn[1:-1,1:-1]**2 -\
                       dt*(F+k)*Vn[1:-1,1:-1]
                
        #Neumann conditions:
        U[0,:] = U[1,:]
        U[-1,:] = U[-2,:]
        U[:,0] = U[:,1]
        U[:,-1] = U[:,-2]
        
        V[0,:] = V[1,:]
        V[-1,:] = V[-2,:]
        V[:,0] = V[:,1]
        V[:,-1] = V[:,-2]
        
        
    return U,V

In [84]:
Ui = U.copy()
Vi = V.copy()

U_8000,V_8000 = ftcs(Ui,Vi,nt,dh,dt,Du,Dv,F,k)

In [96]:
N1 = U_8000[100,::40][0] # First number
print('The first number is: {:.4f}'.format(N1))

N2 = U_8000[100,::40][1] # First number
print('The second number is: {:.4f}'.format(N2))

N3 = U_8000[100,::40][2] # First number
print('The third number is: {:.4f}'.format(N3))

N4 = U_8000[100,::40][3] # First number
print('The fouth number is: {:.4f}'.format(N4))

N5 = U_8000[100,::40][4] # First number
print('The fifth number is: {:.4f}'.format(N5))

The first number is: 0.9247
The second number is: 0.8501
The third number is: 0.6682
The fouth number is: 0.9020
The fifth number is: 0.9040


In [95]:
for i in range(4)
Nx[i] = numpy.where(U_8000[100,::40][i])
print('The {} number ')